# Student allocation to projects

## Getting started
Create 2 csv files or amend them with actual data
- `student_preferences.csv` with 6 columns: student, preference1, preference2, preference3, preference4, preference5
- `project_capacities.csv` with 3 columns: project, capacity, supervisor

Then run every cell, the output are two csv files
- `matching_student.csv` for a student focused view with 2 columns: student, project allocated, preference rank of allocation, list of preferences
- `matching_project.csv` for a project focused view with 4 columns: project, list of student allocated, capacity, supervisor

## Other functions
- `check_matching` checks the validity
- `shuffle_students` randomly moves the order of the student. The order of students will affect the allocations, the student at the top will get largest priority of their choice of projects. They will, by definition, get their first choice.

In [1]:
import pandas as pd
import numpy as np

# Define student and project classes
class Student:
    def __init__(self, id, preferences):
        self.id = id
        self.preferences = preferences
        self.offers = []
        self.matched_project = None
        self.rank_choice = 0

class Project:
    def __init__(self, id, capacity):
        self.id = id
        self.capacity = capacity
        self.matched_students = []
        self.applicant_students = []
        self.rejected_students = []

# Define functions to help algorithm
def assign_project_to_student(student, project):
    project.matched_students.append(student)
    student.matched_project = project
    # student.rank_choice = 1 + student.preferences.index(project.id)

def remove_student_from_project(student, project):
    project.matched_students.remove(student)
    student.matched_project = None

# Function to check validity of matching
def check_matching(students, projects):
    # check no student is matched to more than one project
    for s in students:
        if s.matched_project != None:
            for p in projects:
                if p.id != s.matched_project and p == s.matched_project:
                    print(f"Major Warning - Student {s.id} is matched to multiple projects")
                    return False
            
    # check no project is matched to more than its capacity
    for p in projects:
        if len(p.matched_students) > p.capacity:
            print(f"Major Warning - Project {p.id} is matched to more students than its capacity")
            return False
        
    # warning - check if projects were preferred by students
    for s in students:
        if s.matched_project != None and s.matched_project not in s.preferences:
            print(f"Minor Warning - Student {s.id} is matched to a project they did not prefer")
    
    # warning - check any unassigned students
    if any(s.matched_project == None for s in students):
        print(f"Minor Warning - Some students are not matched to any project")
        
    # valid
    print("Matching is valid")
    return True

# Function to shuffle students
def shuffle_students(students):
    # shuffle students
    np.random.shuffle(students)

# rate matching
def rate_matching(students):
    score = 0
    # rate matching
    for s in students:
        if s.matched_project != None:
            score += 10 * s.rank_choice **-2
    
    return score


In [2]:
# Gale-Shapley Algorithm implementation
def gale_shapley(students, projects, n_prefs=5):

    # Initialise all students and projects as unmatched
    # Initialise applicants for each project (assume project students ranked in order of preference for project)
    for student in students:
        student.matched_project = None
    for project in projects:
        project.matched_students = []
        project.applicant_students = [s for s in students if project.id in s.preferences[:n_prefs]]

    # stop while loop when every project either reaches capacity or has no more applicants
    continue_condition = True

    round_count = 0

    while continue_condition:
        round_count += 1
        # print(f"\nround {round_count} =====================")
        # projects propose to the top student who applied to them
        for p in projects:
            
            if (len(p.applicant_students) !=0 ) & (len(p.matched_students) < p.capacity):
                s = p.applicant_students.pop(0)
                s.offers.append(p.id)
                assign_project_to_student(s, p)

        
        # each student accepts best offer and rejects the rest
        for s in students:
            # print(f"Student {s.id} offers: {s.offers}")
            if s.offers:
                s.offers.sort(key=lambda x: s.preferences.index(x))
                best_offer = s.offers[0]
                for offer in s.offers:
                    if offer != best_offer:
                        project = next(p for p in projects if p.id == offer)
                        project.rejected_students.append(s)
                        remove_student_from_project(s, project)
                s.offers = [best_offer]

        # check if any projects have capacity and unseen applications
        any_unseen_applications = np.array([len(p.applicant_students) > 0 for p in projects])
        any_project_with_capacity = np.array([p.capacity > len(p.matched_students) for p in projects])
        continue_condition = any(any_unseen_applications & any_project_with_capacity)
        
        if round_count > (len(students) * len(projects)):
            break

        # clean up
        for student in students:
            if student.offers:
                student.matched_project = student.offers[0]
                student.rank_choice = 1 + student.preferences.index(student.matched_project)

In [3]:
# Example data - using csv

student_preferences = pd.read_csv("student_preferences.csv")
# 6 columns: student, preference1, preference2, preference3, preference4, preference5

project_capacities = pd.read_csv("project_capacities.csv")
# 3 columns: project, capacity, supervisor

# By default, we will consider all given preferences
N_PREFERENCES_TO_CONSIDER = 5

students = []
projects = []

for index, row in student_preferences.iterrows():
    students.append(Student(row["student"], row[1:N_PREFERENCES_TO_CONSIDER+1].tolist()))

for index, row in project_capacities.iterrows():
    projects.append(Project(row["project"], row["capacity"]))


In [4]:
# Run the algorithm
gale_shapley(students, projects)

# Print the results
print("==========")
print("Student Assignments:")
for student in students:
    print(f"Student {student.id} matched to Project {student.matched_project}")

print("\nProject Assignments:")
for project in projects:
    print(f"Project {project.id} matched to students: {[s.id for s in project.matched_students]}")

Student Assignments:
Student 1 matched to Project Project1
Student 2 matched to Project Project10
Student 3 matched to Project Project4
Student 4 matched to Project Project12
Student 5 matched to Project Project11
Student 6 matched to Project Project3
Student 7 matched to Project Project21
Student 8 matched to Project Project14
Student 9 matched to Project Project6
Student 10 matched to Project Project19
Student 11 matched to Project Project20
Student 12 matched to Project Project5
Student 13 matched to Project Project5
Student 14 matched to Project Project22
Student 15 matched to Project Project7
Student 16 matched to Project Project13
Student 17 matched to Project None
Student 18 matched to Project None
Student 19 matched to Project Project2
Student 20 matched to Project Project7

Project Assignments:
Project Project1 matched to students: [1]
Project Project2 matched to students: [19]
Project Project3 matched to students: [6]
Project Project4 matched to students: [3]
Project Project5

In [5]:
print(f'Matching score: {rate_matching(students)}')

Matching score: 137.36111111111111


# Check validity and export

In [6]:
# check validity of matching
check_matching(students, projects)

Minor Warning - Some students are not matched to any project
Matching is valid


True

In [7]:
# save as pandas
matching_student_df = pd.DataFrame(columns=["student", "project", "rank_choice", "preference"])
for i in range(len(students)):
    student = students[i]
    if type(student.matched_project) is str:
        matching_student_df.loc[i] = [student.id, student.matched_project, student.rank_choice, student.preferences]
    else: 
        # not sure what bug with manual allocation function
        matching_student_df.loc[i] = [student.id, None, student.rank_choice, student.preferences]

matching_project_df = pd.DataFrame(columns=["project", "students", "capacity"])
for i in range(len(projects)):
    project = projects[i]
    matching_project_df.loc[i] = [project.id, [s.id for s in project.matched_students], project.capacity]

# join project_capacities
matching_project_df = matching_project_df.merge(project_capacities[["project","supervisor"]], left_on="project", right_on="project", how="left")

# # export to csv
matching_student_df.to_csv("matching_student.csv", index=False)
matching_project_df.to_csv("matching_project.csv", index=False)

## Evaluation of allocations

In [8]:
matching_student_df

,student,project,rank_choice,preference
0,1,Project1,1,"[Project1, Project20, Project3, Project21, Pro..."
1,2,Project10,1,"[Project10, Project1, Project4, Project3, Proj..."
2,3,Project4,1,"[Project4, Project3, Project19, Project21, Pro..."
3,4,Project12,1,"[Project12, Project21, Project13, Project20, P..."
4,5,Project11,1,"[Project11, Project1, Project14, Project1, Pro..."
5,6,Project3,1,"[Project3, Project22, Project13, Project3, Pro..."
6,7,Project21,2,"[Project12, Project21, Project4, Project21, Pr..."
7,8,Project14,1,"[Project14, Project17, Project2, Project1, Pro..."
8,9,Project6,1,"[Project6, Project8, Project10, Project22, Pro..."
9,10,Project19,1,"[Project19, Project20, Project1, Project21, Pr..."


In [9]:
# total matched students
f'Total matched students: {len(matching_student_df[~matching_student_df["project"].isnull()])}'

'Total matched students: 18'

In [10]:
f'Students without projects: {len(matching_student_df[matching_student_df["project"].isnull()])}'

'Students without projects: 2'

In [11]:
f'Unmatched projects: {len(matching_project_df[(matching_project_df["students"].apply(len) == 0) & (matching_project_df["capacity"]>0)])}'

'Unmatched projects: 12'

In [12]:
matching_student_df.groupby('rank_choice').student.count()

rank_choice
0     2
1    13
2     2
3     1
4     2
Name: student, dtype: int64

In [13]:
# empty projects
matching_project_df[(matching_project_df["students"].apply(len) == 0) & (matching_project_df["capacity"]>0)]

,project,students,capacity,supervisor
7,Project8,[],3,B
8,Project9,[],1,C
14,Project15,[],1,D
15,Project16,[],1,D
16,Project17,[],1,E
17,Project18,[],3,E
22,Project23,[],1,G
23,Project24,[],1,G
24,Project25,[],1,G
25,Project26,[],1,H


In [14]:
# projects with remaining capacity
matching_project_df[(matching_project_df["capacity"] - matching_project_df["students"].apply(len)) > 0]

,project,students,capacity,supervisor
7,Project8,[],3,B
8,Project9,[],1,C
14,Project15,[],1,D
15,Project16,[],1,D
16,Project17,[],1,E
17,Project18,[],3,E
19,Project20,[11],2,E
22,Project23,[],1,G
23,Project24,[],1,G
24,Project25,[],1,G
